# Data Mining - Handin 1 - Clustering 
Welcome to the handin on clustering algorithms and outlier detection. 
This handin corresponds to the topics in Week 5--9 in the course.

The handin IS 
* done individually
* worth 10% of the final grade

For the handin, you will prepare a report in PDF format, by exporting the Jupyter notebook. 
Please submit
1. The jupyter notebook file with your answers
2. The PDF obtained by exporting the jupyter notebook

Submit both files on Brightspace no later than **March 8th kl. 8.59**.

**The grading system**: Tasks are assigned a number of points based on the difficulty and time to solve it. The sum of
the number of points is 100. For the maximum grade you need to get at least _80 points_. The minimum grade (02 in the Danish scale)
requires **at least** 30 points, with at least 8 points on of the first three Parts (Part 1,2,3) and 6 points in the last part (Part 4).
Good luck!

In [ ]:
import sys
#!conda install --yes --prefix {sys.prefix} seaborn

In [ ]:
## DO NOT TOUCH
import numpy as np
import pandas as pd
import warnings
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
import time
import seaborn as sns

RANDOM_SEED = 132414
## DO NOT TOUCH
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt

poke = pd.read_csv("./data/Pokemon.csv")
toy = poke[poke['Type_1'].isin(['Fire', 'Ice'])].sample(n=12, random_state=RANDOM_SEED)

# Part 1 Theoretical Questions

## Task 1.1 K-Means and DBScan

### Task 1.1.1 (2 points)
Give the data set below:
Compute cluster assignments using k-means and k = 2, with initial centroids being (-1, -1) and (2,2)

In [ ]:
color_map = {'Ice':'Blue', 'Fire':'Red'}
X_kmeans = toy[["Attack", "Defense"]]

scaler = StandardScaler().fit(X_kmeans)
X_scaled = scaler.transform(X_kmeans)

plt.scatter(X_scaled[:, 0], X_scaled[:, 1], alpha=0.8, c=toy['Type_1'].map(color_map))
plt.axis('equal');

### Task 1.1.2 (1 point)
Show two examples with two different initial cluster assignments that lead to a different result.

### Task 1.1.3 (4 points)
Compute the dendrograms using single-link

### Task 1.1.4 (2 points)
Run density-based clustering with $\epsilon=1$ and $MinPts=2$. What clusters do you obtain? **For this exercise you can use the DBSCAN from sklearn to check your results** 

## Task 1.2 Elliptic data set (4 points)
After looking at the dataset below, you realize that the clusters are elliptic rather than spherical. You want to detect the red outlier point, assuming you know that is an outlier. 

Which approach would be the most obvious to find the red outlier? Please check the box and motivate your answer below:
- [ ] Distance based approach
- [ ] Angle based approach
- [ ] Depth based approach

**Your answer**

In [ ]:
D_new = np.array([[1.42, 1.131], # Red 
                [1.34, 0.3],
                [1.44, 0.30],
                [1.5, 0.5],
                [1.48, 0.65],
                [1.37, 0.66],
                [1.3, 0.50],
                 ])

plt.scatter(D_new[:, 0], D_new[:, 1], alpha=0.8, c = ['red' if i == 0 else 'blue' for i in range(len(D_new))])
plt.axis([1, 2, 0,2])
plt.show()

## Task 1.3 Theoretical questions (4 points)
1. You are given a measure $d(x,y) = |x-y|$, show that the measure is a metric 
2. Show that $\hat{\Sigma}=
\frac{1}{n}\sum_{i=1}^n (x_i -\hat{\mu}^\top)\cdot(x_i -\hat{\mu}^\top)^\top=E[(X-\hat{\mu})(X-\hat{\mu})^\top]$

**YOUR ANSWER**


# Part 2 Exploratory data analysis
In this section, you will perform preliminary analyses on your data. These preliminary analysis are useful to understand how the data behaves, before running complex algorithms. 

In [ ]:
data = poke.drop(columns = ['#', 'Name', 'Type_1', 'Type_2', 'Generation', 'Legendary'])
data_np = poke.drop(columns = ['#', 'Name', 'Type_1', 'Type_2', 'Generation']).to_numpy()
headers = ["Name", "Type_1", "Type_2", "Total", "HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed", "Generation", "Legendary"]
X = data_np[:,:6]
y = data_np[:,7]
y = y.astype(int) - 1
rows, cols = np.shape(X)
poke.head()

## Task 2.1 Correlation matrix
### Task 2.1.1 (6 points)
Compute the correlation matrix (not covariance matrix) among all the attributes in the code box below

In [ ]:
X = data_np
print("your code here")

### Task 2.1.2 (2 points)
We can also plot the correlation matrix as seen below. What is the relationship between the correlation matrix and the covariance matrix? Please check the correct box below. 

- [ ] The correlation matrix contains the unnormalized covariance values
- [ ] The correlation matrix contains the normalized covariance values
- [ ] The covariance matrix contains the variance of the correlation

In [ ]:
sns.heatmap(poke.corr(),annot=True,linewidths=.5, cmap="YlGnBu", annot_kws={"fontsize":8}, vmax=1)
plt.title('Correlation')
plt.show()

### Task 2.1.3 (6 points)
In this task, we reason about the covariance matrices. Normalize the features using standard score normalization and range normalization. Plot the **covariance** matrices for
1. The unnormalized data
2. The [standard score normalized features](https://en.wikipedia.org/wiki/Standard_score)
3. The range (min-max) normalized features

In [ ]:
X = data_np

print("Your code here!")

### Task 2.1.4 (6 points)
Note how the covariance matrix changes with different normalization schemes and reason on why such behaviour appears.
You should notice some differences. Why are these differences appearing?


- [ ] Range normalization preserves the variance. Therefore, features are directly comparable.
- [ ] Standard score normalization preserves the variance. Therefore, features are directly comparable.
- [ ] Both methods normalize in such a way, that it makes sense to compare the different covariance values to each other.
- [ ] None of the methods normalize in such a way that it makes sense to compare the different covariance values to each other.


## Task 2.2 Normal distribution

### Task 2.2.1 (6 points)
Sometimes it is convenient to know whether a variable is close to a normal distribution. Implement a method norm_dist that: 
    1) Inputs the number of buckets $b$ and a vector $x$ of values
    2) Outputs the sum of the absolute differences of the buckets between the a histogram with $b$ buckets of a normal variable with $\mu,\sigma$ deviation corresponding sample mean and standard deviation of the input vector and the histogram of the input vectors with $b$ buckets. The sum of the differences is computed as 
    $$\sum_{i=1}^b |H_X(i) - H_{\mathcal{N}}(i)|$$ 

Where $H_X(i)$ is the i-th bucket of the histogram of $x$ and $H_\mathcal{N}(i)$ is the i-th bucket of the hisotgram obtained from the normal distribution $\mathcal{N}(\mu,\sigma^2)$. 

Use the norm function from Scipy to get the normal distribution to subtract from.

In [ ]:
from scipy.stats import norm

## Our data comes from the variable X
## print(X)
X = data_np
def norm_dist(x, b): 
    dist = 0
    ### YOUR CODE HERE
    
    ### YOUR CODE HERE
    return dist

### Task 2.2.2 (1 point)
Is the method in Task 2.2.1 a good method? Is it robust to outliers? Run your code on each columns of the dataset.
What is the column with the largest distance? Compare the norm_dist of each attribute feature in the dataset.

In [ ]:
# Our data is still from X

print("YOUR CODE HERE")

### Task 2.2.3 (2 points)
Now look at the method below. This is called a Quantile-Quantile [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). 

Discuss why this method is more robust than the one we proposed in Task 2.2.1.

**YOUR ANSWER HERE**

In [ ]:
from scipy import stats
from matplotlib import gridspec

plt.tight_layout()
_, n = data.shape

fig = plt.figure(constrained_layout=True, figsize=(8, 30))
spec = gridspec.GridSpec(ncols=2, nrows=(n-1), figure=fig)
for i in np.arange(3,n): 
    x = data[headers[i]]
    r = i-1
    qq = fig.add_subplot(spec[r, 1]) 
    stats.probplot(x, plot=qq)
    h = fig.add_subplot(spec[r, 0])
    h.set_title(headers[i])
    h.hist(x, bins = 30)

# Part 3 Cluster Analysis
In this section, you will perform cluster analysis of the dataset above and modify clustering algorithms to achieve better results. 

## Task 3.1

### Task 3.1.1 (6 points)
Implement and use the elbow method detect the number of clusters . For plotting you can use the **silhouette coefficient.**

Use the "Total" and "Stp. Atk" features of the data set.

In [ ]:
X = toy[["Total", "Sp. Atk"]].to_numpy()
print("YOUR CODE HERE")

**YOUR TEXT HERE**

### Task 3.1.2 (2 points)
Run k-means on the dataset X, with the number of clusters detected in the previous exercise.

**Note**: you can use the KMeans implementation from scikit-learn

In [ ]:
### YOUR CODE HERE
X = toy[["Total", "Sp. Atk"]].to_numpy()

# Necessary Data normalization!
X_norm = (X - X.min(0)) / X.ptp(0)

clusters = []

plt.scatter(X_norm[:, 0], X_norm[:, 1], alpha=0.8, c=clusters)

### Task 3.1.3 (4 points)
Implement Kernel K-means and the Gaussian Kernel. 

The Gaussian kernel is defined as in the following equation:

$$
K\left(\mathbf{x}_{i}, \mathbf{x}_{j}\right)=\exp \left(-\frac{\left\|\mathbf{x}_{i}-\mathbf{x}_{j}\right\|^{2}}{2 \sigma^{2}}\right)$$

In [ ]:
### YOUR CODE HERE
X = toy[["Total", "Sp. Atk"]].to_numpy()

# Necessary Data normalization!
X_norm = (X - X.min(0)) / X.ptp(0)

def gaussian_kernel(x, y, sigma=0.8): 
    k = 0 
    ### YOUR CODE HERE


    ### YOUR CODE HERE
    return k


def kernel_kmeans(X, n_clusters, kernel=gaussian_kernel, iters=100, error=.01): 
    ### YOUR CODE HERE


    ### YOUR CODE HERE
    return clusters


clusters = kernel_kmeans(X_norm, 3)

### Task 3.1.4 Visualise the results (0 points)
The visualization below uses a method called [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) to plot the data into a 2D space. 
The method is not part of the course, but it is a good tool for data analysis that should be always considered. 

In [ ]:
## DO NOT MODIFY
def plot_clusters(X, clusters): 
    time_start = time.time()
    tsne = TSNE(n_components=2, verbose=1, random_state=0)
    X_2d = tsne.fit_transform(X)
    print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

    from matplotlib import pyplot as plt
    plt.figure(figsize=(6, 5))
    colors = ['b','r','g', 'm', 'c', 'y', 'k', 'tan', 'gold','sienna', 'navy','darkgreen']
    for i in np.arange(np.shape(data)[0]):
        plt.scatter(X_2d[i, 0], X_2d[i, 1], c=colors[clusters[i]])
    plt.show()

In [ ]:
plot_clusters(X_norm, y)
plot_clusters(X_norm, clusters)

## Task 3.2 Clustering quality

### Task 3.2.1 (4 points)
Now we will implement, *mutual information* and a measure for clustering quality.

Entropy for a clustering is $H(C) = - \sum_{i=1}^{k}{p_{C_i} \log {p_{C_i}}}$.

Mutual information for two clusterings $U$ and $V$ is defined as $\text{MI}(U,V) = -\sum\limits^{|U|}_{i=1} \sum\limits^{|V|}_{j=1} \frac{|U_i \cap V_j|}{N} \log  N \frac{|U_i \cap V_j|}{|U_i||V_j|}$.
We first need to implement entropy and then we will define mutual information.

In [ ]:
def entropy(C):
    # Let C be a list of clusters
    entropy = 0
    ### YOUR CODE HERE
    
    
    ### YOUR CODE HERE
    return entropy


def MI(C1, C2):
    mi = 0
    ### YOUR CODE HERE
    
    
    ### YOUR CODE HERE
    return mi

def NMI(C1, C2):
    mi = MI(C1, C2)
    h_c1 = entropy(C1)
    h_c2 = entropy(C2)
    return mi/(np.sqrt(h_c1 * h_c2))
    

X = toy[["Total", "Sp. Atk"]].to_numpy()

# Necessary Data normalization!
X_norm = (X - X.min(0)) / X.ptp(0)

### Task 3.2.2 (4 points)
Plot the NMI (Use the provided NMI function from the previous task) among the class labels $y$ and the clusters you found with k-means in Task 3.1. 
- Reason about the measure, is the measure influenced by the size of the clusters?  
- What does the measure capture? 

In [ ]:
###YOUR CODE HERE


### Task 3.2.3 (4 points)
Provide an implementation of $purity_i = \frac{1}{|C_i|} \max_{j=1..k}\{n_{ij}\}$ where $n_{ij}$ is the number of common points in cluster $$C_i$$
and ground-truth clusters obtained from the labels $y$.

In [ ]:
### YOUR CODE HERE
T = np.array([]) # Ground-truth clusters
C = np.array([]) # Clusters obtained by k-means
### YOUR CODE HERE

## C is the clustering from k-means and T is the ground truth cluster assignments.
def purity(C, T):
    purity = 0
    ### YOUR CODE HERE


    ### YOUR CODE HERE
    return purity

print('Purity: {}, NMI: {}'.format(purity(C,T), NMI(C,T)))

### Task 3.2.4 (2 points)

Plot purity and compare purity with NMI. Which measure is preferable?

- [ ] NMI is preferable because it uses all the points
- [ ] Purity is preferable because it is less computational demanding
- [ ] NMI is preferrable because it does not favor small clusters
- [ ] Purity is preferrable because it tends to favor balanced clusters.


### Task 3.3 DBScan


### Task 3.3.1 (1 point)

Run DB-Scan with parameters $\varepsilon=0.5, minPts=3$ from sklearn and compare the results with k-means. Which of the two achieve a better NMI? 


In [ ]:
### YOUR ANSWER

dbscan = DBSCAN(eps=0.5, min_samples=3).fit(X_norm)
clusters_dbs = dbscan.labels_
plot_clusters(X_norm, clusters_dbs)

### Task 3.3.2 (4 points)
DBScan requires tuning of parameters $\varepsilon, MinPts$. Implement the heuristic strategy in the slides to find the best parameters. Run the code and see whether the results with DBScan improve.

In [ ]:
def tune_dbscan(X): 
    eps = 0
    min_pts = (X.shape[1] *2)-1
    ### YOUR CODE HERE
    
    
    ### YOUR CODE HERE
    return eps

### Task 3.3.3 (6 points)
In this last point, we will try to implement a simple subspace clustering algorithm and compare the results with CLIQUE implemented in Week 8.
1. Take all subsets of 2,3 attributes from the variable *toy*. Beware that you should only use the numerical attributes.
2. Run dbscan on each subset. 
3. Compute NMI for each subset. 
4. Keep the k subsets with the largest NMI. 
    
**Note**: You may have to experiment a lot with eps and MinPts to get reasonable clusterings


In [ ]:
print("YOUR CODE HERE!")

### Task 3.3.4 (4 points)
Analyze the following from Task 3.3.3
1. Advantages and disadvantages of the proposed algorithm
2. Results with respect to the CLIQUE algorithm

**YOUR ANSWER HERE**

# Part 4 Outlier detection
In this exercise we will work with outlier detection techniques and analyze their performance on the small dataset.

In [ ]:
X_small = toy[["Attack", "Defense"]]

scaler = StandardScaler().fit(X_small)
X_scaled = scaler.transform(X_small)

## Task 4.1 (DBoutliers)
We will now compare two outlier detection techniques.
### Task 4.1.1 (6 points)
We will first implement a simple distance-based outlier detector. This is the distance-based outlier detection from the lectures, where a point is considered an outlier if at most a fraction pi of the other points have a distance less of than eps to it.

In [ ]:
def DBOutliers(X, eps, pi): 
    outliers = None
    ### YOUR CODE HERE
    
    
    ### YOUR CODE HERE
    return outliers

## Task 4.1.2 (2 points)
DBOutliers requires tuning the parameters eps, pi. Discuss how the results vary with those parameters. 

In [ ]:
### YOUR CODE

**YOUR ANSWER**

### Task 4.1.3 (3 points)
**NOTE** This is hard but also fun. Since it is not impacting the grade too much, you can choose to invent something new.

Propose a heuristic method to tune parameters eps, pi. 

In [ ]:
def tune_dboutliers(X): 
    eps = 0
    pi = 0
    ### YOUR CODE HERE
    
    
    ### YOUR CODE HERE
    return eps, pi

## Task 4.2 LOF (2 points)
Using the parameters eps=0.9, pi=0.2 (and using the normalized data-set) compare the results of DBOutliers with those of
LOF implemented in Week 9. What outliers do you find?

In [ ]:
print("Your code here!")

**YOUR ANSWER**



